In [ ]:
# Mainly for Dt_AD to check why the larger uncertainty and 
# to check if it actually depends on the center_y_lens0 parameter

In [ ]:
import argparse

parser = argparse.ArgumentParser(description="Produces the MCMC posterior and corner plots of SOME parameters, Dt in particular")
parser.add_argument("-c", "--cut_mcmc", type=int, dest="cut_mcmc", default=0,
                    help="cut the first <c> steps of the mcmc to ignore them")
parser.add_argument('SETTING_FILE', help="setting file to consider")
args = parser.parse_args()

setting_name =  args.SETTING_FILE.replace(".py","")
cut_mcmc = args.cut_mcmc

In [ ]:
import numpy as np
import json
from scipy import stats
import os,sys
from datetime import datetime
import time 
import pickle
import importlib
import corner
import matplotlib.pyplot as plt
import copy

####################
from tools import *
present_program(sys.argv[0])
####################
backup_path="./backup_results/"


savemcmc_path="./"+backup_path+"/"+setting_name.replace("settings_","mcmc_")+"/"
savefig_path="./"+backup_path+"/"+setting_name.replace("settings_","")+"/"
mkdir(savefig_path)

mcmc_file=savemcmc_path+setting_name.replace("settings","mcmc_smpl")+".json"
mcmc_fermat_file=savemcmc_path+setting_name.replace("settings","mcmc_ordered_fermat")+".json"
param_file=savemcmc_path+setting_name.replace("settings","mcmc_prm")+".dat"
   
#MCMC sample
with open(mcmc_file, 'r') as f:
    samples_mcmc = np.array(json.load(f)[cut_mcmc:])

#MCMC sample for Fermat pot
#NOTE: they are ordered A,B,C,D
with open(mcmc_fermat_file, 'r') as f:
    mcmc_fermat = np.array(json.load(f)[cut_mcmc:])


#parameters' name
param_mcmc=[]
with open(param_file,"r") as param_file:
    param_mcmc=(param_file.readlines())
for i in range(len(param_mcmc)):
    param_mcmc[i]=param_mcmc[i].replace(",\n","")


In [ ]:
# Time delay behaviour
########################
param_mcmc_t = ["td_A","td_B","td_C","td_D"]

import numpy as np
from Dt_from_Df import Dt_XY
from astropy import units as u
H0_planck = 67.4*u.km/u.second/u.megaparsec # km/s/Mpc

In [ ]:
setting_dir = find_setting_path(setting_name)
sys.path.append(setting_dir)
setting_module = importlib.import_module(setting_name)
setting = setting_module.setting()

In [ ]:
mcmc_t = Dt_XY(Df_XY=mcmc_fermat,H0=H0_planck,z_l=setting.z_lens,z_s=setting.z_source)
# order A, B, C, D

In [ ]:
new_samples = copy.copy(samples_mcmc.T)
for t in mcmc_t.T:
    new_samples = np.array([*new_samples,t])
param_mcmc = param_mcmc + param_mcmc_t

In [ ]:
red_samples = []
red_param = []
for sample_i,param_i in zip(new_samples,param_mcmc):
    if "td_" in param_i or "center_" in param_i or "ra_" in param_i or "dec_" in param_i:
        red_samples.append(sample_i)
        prm_i = param_i
        if "center_" in param_i:
            prm_i = setting.str_param(param_i)
        red_param.append(prm_i)
red_samples = np.array(red_samples).T

plot= corner.corner(red_samples, labels=red_param,show_titles=True)
#plot.title("$H_0^{Planck} = $"+str(H0_planck))

plot.savefig(savefig_path+"/t_corner_red.png")

In [ ]:
# consider also the Dt plot (alone, but now with real Deltas)
mcmc_t = np.array(mcmc_t)
Delta_T = np.transpose(mcmc_t.T[1:]-mcmc_t.T[0]).tolist()
labels = ["$\Delta t_{A-B}$","$\Delta t_{A-C}$", "$\Delta t_{A-D}$"]
plot= corner.corner(np.array(Delta_T), labels=labels,show_titles=True)
#plot.title("$H_0^{Planck} = $"+str(H0_planck))
plot.savefig(savefig_path+"/Dt_corner.png")

In [ ]:
success(sys.argv[0])